In [ ]:
import numpy as np
import tensorflow as tf
import _pickle as pickle


In [ ]:
# player 0, player 1
array_player_1 = np.full((1000, 3, 3), 0, dtype=np.float32)
array_player_2 = np.full((1000, 3, 3), 0, dtype=np.float32)
array_player_1[0] = [[1, 0, 0], [1, 0, 1], [1, 1, 0]]
array_player_2[0] = [[0, 1, 1], [0, 1, 0], [0, 0, 1]]
# multi class label
# [1, 0] player 1 wins
# [0, 1] player 2 wins
# [0, 0] draw
outcome = np.full((1000, 2), 0, dtype=np.float32)
outcome[0] = [1, 0]

In [ ]:
array_player_2 

In [ ]:
# tf estimator
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"player1": array_player_1, "player2": array_player_2},
    y=outcome,
    num_epochs=1,
    batch_size=128,
    shuffle=False) 


In [ ]:
def a_model_fn(features, labels, mode, params):
    p1 = tf.reshape(features['player1'], [-1, 9])
    p2 = tf.reshape(features['player2'], [-1, 9])
    players = tf.concat([p1, p2], axis=1)
    print(players.get_shape())
    hidden = tf.layers.dense(players, 24)
    predictions = tf.layers.dense(players, 2)

    loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=labels, logits=predictions)
    # actual optimization functions
    optimizer = tf.train.AdamOptimizer(1e-4, epsilon=1e-6)
    train_op = optimizer.minimize(
        loss=loss, global_step=tf.train.get_global_step())
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels, predictions),
    }
    return tf.estimator.EstimatorSpec(
        mode=mode,
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops)

In [ ]:
nn = tf.estimator.Estimator(model_fn=a_model_fn, params={}, model_dir="/tmp/tictacfow_practice")

In [ ]:
nn.train(input_fn=train_input_fn, steps=160)


In [ ]:
# more thoughts on data formatting
array_player_1 = np.full((1000, 3, 3), -2)
array_player_2 = np.full((1000, 3, 3), -2)

In [ ]:
for i in range(1000):
    array_player_1[i] = [[1, 0, 0], [1, 0, 1], [1, 1, 0]]
    array_player_2[i] = [[0, 1, 1], [0, 1, 0], [0, 0, 1]]


In [ ]:
the_data = {'player1': array_player_1, 'player2': array_player_2, 'y': outcome}
with open(filein, 'wb') as f:
    preview = pickle.dump(the_data, f)